In [301]:
from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import cv2
import pandas as pd

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils
from keras_frcnn.mylayer import mylayer
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

K.set_learning_phase(1) #set learning phase

# sys.setrecursionlimit(40000)

In [302]:
network ='vgg'

if network == 'vgg':
	from keras_frcnn import vgg as nn
elif network == 'resnet50':
	from keras_frcnn import resnet as nn
else:
	print('Not a valid model')
	raise ValueError


In [303]:
from keras_frcnn.RoiPoolingConv import RoiPoolingConv
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout

if K.image_dim_ordering() == 'th':
	input_shape_img = (3, None, None)
else:
	input_shape_img = (400, 400, 3)

img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(6, 4))
nb_classes = 575

#pdb.set_trace()
num_rois = 6
# compile times on theano tend to be very high, so we use smaller ROI pooling regions to workaround

if K.backend() == 'tensorflow':
    pooling_regions = 7
    input_shape = (num_rois,7,7,512)
elif K.backend() == 'theano':
    pooling_regions = 7
    input_shape = (num_rois,512,7,7)

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)


out_roi_pool = mylayer(pooling_regions, 6)([shared_layers, roi_input])
print out_roi_pool

out = (Flatten(name='flatten'))(out_roi_pool)
out = (Dense(4096, activation='relu', name='fc1'))(out)
out = (Dropout(0.5))(out)
out = (Dense(2048, activation='relu', name='fc2'))(out)
out = (Dropout(0.5))(out)
print 'no of classes', nb_classes
out_class = (Dense(73, activation='sigmoid', kernel_initializer='zero'))(out)

model_classifier = Model([img_input, roi_input], out_class)


try:
	print('loading weights from {}'.format(C.base_net_weights))
	model_classifier.load_weights(C.base_net_weights, by_name=True)
except:
	print('Could not load pretrained model weights. Weights can be found in the keras application folder \
		https://github.com/fchollet/keras/tree/master/keras/applications')

optimizer = Adam(lr=1e-5)
optimizer_classifier = Adam(lr=1e-5)
model_classifier.compile(optimizer=optimizer_classifier, loss = 'binary_crossentropy', metrics= ['accuracy'])



Tensor("mylayer_2/Reshape:0", shape=(1, 6, 7, 7, 512), dtype=float32)
no of classes 575
Could not load pretrained model weights. Weights can be found in the keras application folder 		https://github.com/fchollet/keras/tree/master/keras/applications


In [304]:
def name(x):
    name = str.split(x,'/')
    return '/Users/1024329/Downloads/DeepFashion/img_n/' + name[1] + '/' + name[2]
    
df3 = pd.read_pickle('upper_body_atts.pkl')
df3['img_name'] = df3['img_name'].apply(lambda x:name(x))

In [305]:
df3 = df3[['img_name','landmarks','atts']]
df3.head()

,img_name,landmarks,atts
0,/Users/1024329/Downloads/DeepFashion/img_n/Lon...,"[[96, 2, 60, 60], [165, 2, 60, 60], [300, 153,...","[99, 495, 571, 881, 940]"
1,/Users/1024329/Downloads/DeepFashion/img_n/Hoo...,"[[105, 5, 60, 60], [255, 3, 60, 60], [26, 195,...","[470, 953]"
2,/Users/1024329/Downloads/DeepFashion/img_n/Rea...,"[[126, 36, 60, 60], [244, 42, 60, 60], [67, 3,...","[745, 883]"
3,/Users/1024329/Downloads/DeepFashion/img_n/Flo...,"[[67, 8, 60, 60], [217, 2, 60, 60], [24, 167, ...","[142, 306, 365]"
4,/Users/1024329/Downloads/DeepFashion/img_n/Bea...,"[[51, 68, 60, 60], [280, 80, 60, 60], [25, 13,...","[50, 121, 122, 226, 681]"


In [306]:

img = []
landmarks = []
attribs = []
for i in range(30):
    if i%1000 == 0:
        print 'another 1000', i 
    a = cv2.imread(df3.loc[i,'img_name'])
    a = cv2.resize(a,(400,400))
    img.append(a)
    lnd = df3.loc[i,'landmarks']
    landmarks.append(lnd)
    atts = df3.loc[i,'atts']
    attribs.append(atts)

another 1000 0


In [307]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

X = np.array(img)
X2 = np.array(landmarks)
Y = mlb.fit_transform(attribs)
X2 = (X2/4).astype(int)
print 'shapes of input and output', X.shape, X2.shape, Y.shape


shapes of input and output (30, 400, 400, 3) (30, 6, 4) (30, 73)


In [262]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='min', period=1)
# model_classifier.fit(x = [X,X2],y= Y, batch_size=1, epochs=10,validation_split=0.2,callbacks = [checkpoint])